In [32]:
#Aubay Azzarouk 06/03/25 start
#Gen AI - Financial Analysis under the scope of a junior data scientist. 
#Prioritizing modular and reproducible code. 
# -----------------------------------------------------------------------------

# PHASE 1: 
#Setting up env to aid analysis of extracted financial data from the SEC Edgar (10-K filings - Apple, Tesla, Microsoft).


import pandas as pd # for data manipulation
import os  # handles os interactions for paths, directory 
from pathlib import Path # handles input/output locations of file system paths. 

In [33]:
# -----------------------------------------------------------------------------
# PHASE 2: Data Ingestion

# Reads manually extracted financial data (10-K figures) from excel converted to csv into Python for analysis. 
# The dataset contains the last 3 fiscal years for Apple, Tesla, and Microsoft, 
# Key metrics: Revenue, Net Income, Assets, Liabilities, and Cash Flow from Operations.

# -----------------------------------------------------------------------------

# Define the path to the source file (manually extracted from SEC EDGAR filings).
    # IMPORTANT: Adjust this path if running on a different machine or directory.
xlsx_path = 'Downloads/10K_Financial_Metrics_GenAI_BCG.csv'

# Define an output directory for cleaned/processed files, charts, or exports.
# Using pathlib ensures compatibility across operating systems.
out_dir = Path("outputs")

# Create the output directory if it does not already exist.
# exist_ok=True → no error if the folder already exists.
# parents=True → create parent folders automatically if missing.
out_dir.mkdir(exist_ok=True, parents=True)

# load the CSV into a pandas DataFrame.
# load the entire dataset to inspect structure first.
df = pd.read_csv(xlsx_path)
# preview
df.head()




,Company,Year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Ops
0,Microsoft Corp.,2024,"245,122","88,136","512,163","243,686","118,458"
1,Microsoft Corp.,2023,"211,915","72,361","411,976","205,753","87,582"
2,Microsoft Corp.,2022,"198,270","72,738",NaN,NaN,"89,035"
3,"Tesla, inc.",2024,"97,690","7,091","122,070","48,390","14,923"
4,"Tesla, inc.",2023,"96,773","14,997","106,618","43,009","13,256"


In [34]:
# -----------------------------------------------------------------------------
# PHASE 3: Analyzing Financial Trends w/ pd: 
# -----------------------------------------------------------------------------
# Objective: 
#   - Use pd to calculate YoY changes for each metric.

#   - Clean numeric columns (remove commas, convert to floats)
#   - Compute derived metrics:
#         Revenue Growth (%), Net Income Growth (%),
#         Net Profit Margin (%), Operating CF Margin (%), Leverage
#   - Build summary views for latest year and multi-year trends
# -----------------------------------------------------------------------------

In [35]:
#list to hold names of the key metrics in question. 
num_cols = ["Total Revenue", "Net Income", "Total Assets",
            "Total Liabilities", "Cash Flow from Ops"]

#iterate through each column to strip commas and spaces, and then convert to floats.
for col in num_cols:
    df[col] = (
        df[col].astype(str)          # ensure string
              .str.replace(",", "", regex=False)  # strip commas
              .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")  # convert to float



#Sort by alphabetial order and reset index to get 0,1,2 label sequence.
df = df.sort_values(["Company", "Year"]).reset_index(drop=True)
#print(df.head)
#Begin analyzing the financial data across yearly changes. 

# YoY growth (within each company)
# compare revenue and net income by prev and return as a new column with as a change in %.
df["Revenue Growth (%)"] = df.groupby("Company")["Total Revenue"].pct_change() * 100
df["Net Income Growth (%)"] = df.groupby("Company")["Net Income"].pct_change() * 100

# Profitability & leverage ratios
df["Net Profit Margin (%)"]   = (df["Net Income"] / df["Total Revenue"]) * 100
df["Operating CF Margin (%)"] = (df["Cash Flow from Ops"] / df["Total Revenue"]) * 100
df["Leverage (Liab/Assets)"]  = (df["Total Liabilities"] / df["Total Assets"]) #liabilities / assets = leverage financed by debt.


In [36]:
# Latest year summary per company (one row each)
latest_summary = (
    df.sort_values(["Company", "Year"])
      .groupby("Company")
      .tail(1)
      .reset_index(drop=True)
)

# Multi-year trend view (all 2022–2024 rows)
trend_view = df.copy()

#Columns to format
pct_cols = ["Revenue Growth (%)", "Net Income Growth (%)", 
            "Net Profit Margin (%)", "Operating CF Margin (%)", 
            "Leverage (Liab/Assets)"]
dollar_cols = ["Total Revenue", "Net Income", "Total Assets", 
               "Total Liabilities", "Cash Flow from Ops"]

# 4) Create presentation-friendly copies
pretty_latest = latest_summary.copy()
pretty_trend  = trend_view.copy()

#convert financials (in millions → trillions USD), round to 2 decimals
for c in dollar_cols:
    pretty_latest[c] = (pretty_latest[c] / 1000).round(2)   # trillions
    pretty_trend[c]  = (pretty_trend[c] / 1000).round(2)

#round % and ratio columns to 2 decimals
pretty_latest[pct_cols] = pretty_latest[pct_cols].round(2)
pretty_trend[pct_cols]  = pretty_trend[pct_cols].round(2)

#preview and display results
print("=== Latest Year Summary (Trillions USD, % rounded) ===")
display(pretty_latest)

print("\n=== Multi-Year Trend View (Trillions USD, % rounded) ===")
display(pretty_trend)


=== Latest Year Summary (Trillions USD, % rounded) ===


,Company,Year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Ops,Revenue Growth (%),Net Income Growth (%),Net Profit Margin (%),Operating CF Margin (%),Leverage (Liab/Assets)
0,Apple Inc.,2024,391.04,93.74,152.99,308.03,118.25,2.02,-6.26,23.97,30.24,2.01
1,Microsoft Corp.,2024,245.12,88.14,512.16,243.69,118.46,15.67,21.80,35.96,48.33,0.48
2,"Tesla, inc.",2024,97.69,7.09,122.07,48.39,14.92,0.95,-52.72,7.26,15.28,0.40



=== Multi-Year Trend View (Trillions USD, % rounded) ===


,Company,Year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Ops,Revenue Growth (%),Net Income Growth (%),Net Profit Margin (%),Operating CF Margin (%),Leverage (Liab/Assets)
0,Apple Inc.,2022,394.33,99.90,NaN,NaN,122.15,NaN,NaN,25.34,30.98,NaN
1,Apple Inc.,2023,383.28,100.00,143.57,290.44,110.54,-2.80,0.09,26.09,28.84,2.02
2,Apple Inc.,2024,391.04,93.74,152.99,308.03,118.25,2.02,-6.26,23.97,30.24,2.01
3,Microsoft Corp.,2022,198.27,72.74,NaN,NaN,89.04,NaN,NaN,36.69,44.91,NaN
4,Microsoft Corp.,2023,211.92,72.36,411.98,205.75,87.58,6.88,-0.52,34.15,41.33,0.50
5,Microsoft Corp.,2024,245.12,88.14,512.16,243.69,118.46,15.67,21.80,35.96,48.33,0.48
6,"Tesla, inc.",2022,81.46,12.56,NaN,NaN,14.72,NaN,NaN,15.41,18.07,NaN
7,"Tesla, inc.",2023,96.77,15.00,106.62,43.01,13.26,18.80,19.44,15.50,13.70,0.40
8,"Tesla, inc.",2024,97.69,7.09,122.07,48.39,14.92,0.95,-52.72,7.26,15.28,0.40


In [37]:
# Aggregate & Comparative Analysis:
# Purpose: 
    # Summarize average financial performance over 3 years per company. 
    # Rank comapnies by revenue and Net Income each year

#adding derived metrics from YoY change to aggregate into one list, w/out revenue and net income growth
ratio_cols = [ "Net Profit Margin (%)", "Operating CF Margin (%)", "Leverage (Liab/Assets)"]

all_metrics = dollar_cols + ratio_cols 

three_yearavg = (
    df.groupby("Company")[all_metrics].mean(numeric_only=True).reset_index()
)

#three_yearavg.head()
pretty_three_yearavg = three_yearavg.copy()
#format 3 year averages with rounded %: 
for c in dollar_cols:
    pretty_three_yearavg[c] = (pretty_three_yearavg[c]/1000).round(2)

pretty_three_yearavg[ratio_cols] = pretty_three_yearavg[ratio_cols].round(2)



#rank by year: 
df["Revenue Rank"] = df.groupby("Year")["Total Revenue"].rank(ascending=False, method = "dense")
df["Net Income Rank"] = df.groupby("Year")["Net Income"].rank(ascending=False, method = "dense")

rank_view = (
    df[["Company", "Year", "Total Revenue", "Revenue Rank", 
        "Net Income", "Net Income Rank"]]
      .sort_values(["Year", "Revenue Rank"])
      .reset_index(drop=True)
)

#converting financials (millions → trillions, rounded) for reporting
rank_view[["Total Revenue", "Net Income"]] = (rank_view[["Total Revenue", "Net Income"]] / 1000).round(2)

print("\n=== Company Rankings by Year (Revenue & Net Income, Trillions USD) ===")
display(rank_view)
print(" 3-Year Averages per Company (Trillions USD, % rounded)")
display(pretty_three_yearavg)


=== Company Rankings by Year (Revenue & Net Income, Trillions USD) ===


,Company,Year,Total Revenue,Revenue Rank,Net Income,Net Income Rank
0,Apple Inc.,2022,394.33,1.0,99.90,1.0
1,Microsoft Corp.,2022,198.27,2.0,72.74,2.0
2,"Tesla, inc.",2022,81.46,3.0,12.56,3.0
3,Apple Inc.,2023,383.28,1.0,100.00,1.0
4,Microsoft Corp.,2023,211.92,2.0,72.36,2.0
5,"Tesla, inc.",2023,96.77,3.0,15.00,3.0
6,Apple Inc.,2024,391.04,1.0,93.74,1.0
7,Microsoft Corp.,2024,245.12,2.0,88.14,2.0
8,"Tesla, inc.",2024,97.69,3.0,7.09,3.0


 3-Year Averages per Company (Trillions USD, % rounded)


,Company,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Ops,Net Profit Margin (%),Operating CF Margin (%),Leverage (Liab/Assets)
0,Apple Inc.,389.55,97.88,148.28,299.23,116.98,25.13,30.02,2.02
1,Microsoft Corp.,218.44,77.74,462.07,224.72,98.36,35.60,44.85,0.49
2,"Tesla, inc.",91.98,11.55,114.34,45.70,14.30,12.72,15.68,0.40


In [38]:

#Rank by total revenue and net income 

# 1) Calculate ranks (1 = highest)
df["Revenue Rank"]    = df.groupby("Year")["Total Revenue"].rank(ascending=False, method="dense")
df["Net Income Rank"] = df.groupby("Year")["Net Income"].rank(ascending=False, method="dense")

# 2) Ranking view for all companies per year
rank_view = (
    df[["Company", "Year", "Total Revenue", "Revenue Rank", 
        "Net Income", "Net Income Rank"]]
      .sort_values(["Year", "Revenue Rank"])
      .reset_index(drop=True)
)

# Convert financials (millions → trillions for readability)
rank_view[["Total Revenue", "Net Income"]] = (rank_view[["Total Revenue", "Net Income"]] / 1000).round(2)

# 3) Identify top performers per year
top_revenue = (
    df.loc[df.groupby("Year")["Total Revenue"].idxmax(),
           ["Company", "Year", "Total Revenue"]]
    .assign(Metric="Top Revenue")
)

top_net_income = (
    df.loc[df.groupby("Year")["Net Income"].idxmax(),
           ["Company", "Year", "Net Income"]]
    .assign(Metric="Top Net Income")
)

# Convert to trillions for presentation
top_revenue["Total Revenue"] = (top_revenue["Total Revenue"] / 1000).round(2)
top_net_income["Net Income"] = (top_net_income["Net Income"] / 1000).round(2)

# Combine leaderboards
leaderboard = pd.concat([top_revenue, top_net_income]).sort_values("Year")

print("=== Company Rankings by Year (Trillions USD) ===")
display(rank_view)

print("\n=== Yearly Top Performers (Leaders) ===")
display(leaderboard)

=== Company Rankings by Year (Trillions USD) ===


,Company,Year,Total Revenue,Revenue Rank,Net Income,Net Income Rank
0,Apple Inc.,2022,394.33,1.0,99.90,1.0
1,Microsoft Corp.,2022,198.27,2.0,72.74,2.0
2,"Tesla, inc.",2022,81.46,3.0,12.56,3.0
3,Apple Inc.,2023,383.28,1.0,100.00,1.0
4,Microsoft Corp.,2023,211.92,2.0,72.36,2.0
5,"Tesla, inc.",2023,96.77,3.0,15.00,3.0
6,Apple Inc.,2024,391.04,1.0,93.74,1.0
7,Microsoft Corp.,2024,245.12,2.0,88.14,2.0
8,"Tesla, inc.",2024,97.69,3.0,7.09,3.0



=== Yearly Top Performers (Leaders) ===


,Company,Year,Total Revenue,Metric,Net Income
0,Apple Inc.,2022,394.33,Top Revenue,NaN
0,Apple Inc.,2022,NaN,Top Net Income,99.90
1,Apple Inc.,2023,383.28,Top Revenue,NaN
1,Apple Inc.,2023,NaN,Top Net Income,100.00
2,Apple Inc.,2024,391.04,Top Revenue,NaN
2,Apple Inc.,2024,NaN,Top Net Income,93.74


In [39]:
# -----------------------------------------------------------------------------
# PHASE 3 (continued): Add CAGR (2022 → 2024) for Revenue & Net Income
# -----------------------------------------------------------------------------
# CAGR Formula:
#   CAGR = (Final Value / Initial Value) ** (1 / (n_years - 1)) - 1
#   Here: n_years = 3 (2022 → 2024), so exponent = 1/2

#Helper function for CAGR
def calc_cagr(group, col):
    start_val = group.loc[group["Year"] == 2022, col].values
    end_val   = group.loc[group["Year"] == 2024, col].values
    if len(start_val) and len(end_val) and start_val[0] > 0:
        return ((end_val[0] / start_val[0]) ** (1/2) - 1) * 100
    else:
        return float("nan")

#Applying CAGR per company for Revenue & Net Income
cagr_df = (
    df.groupby("Company")
      .apply(lambda g: pd.Series({
          "Revenue CAGR (%)":    calc_cagr(g, "Total Revenue"),
          "Net Income CAGR (%)": calc_cagr(g, "Net Income")
      }))
      .reset_index()
)

#merging CAGR results into 3-year averages summary
three_yearavg_with_cagr = three_yearavg.merge(cagr_df, on="Company")

#Format presentation copy
pretty_three_yearavg_cagr = three_yearavg_with_cagr.copy()
#scaling and rounding %
for c in dollar_cols:
    pretty_three_yearavg_cagr[c] = (pretty_three_yearavg_cagr[c] / 1000).round(2)

#Rounding ratios + CAGR columns
format_cols = ratio_cols + ["Revenue CAGR (%)", "Net Income CAGR (%)"]
pretty_three_yearavg_cagr[format_cols] = pretty_three_yearavg_cagr[format_cols].round(2)

print("=== 3-Year Averages + CAGR per Company (Trillions USD, % rounded) ===")
display(pretty_three_yearavg_cagr)


=== 3-Year Averages + CAGR per Company (Trillions USD, % rounded) ===


/var/folders/v0/cv7w3yvx4kngj01jlbzhzt_r0000gn/T/ipykernel_2098/929883142.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,Company,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow from Ops,Net Profit Margin (%),Operating CF Margin (%),Leverage (Liab/Assets),Revenue CAGR (%),Net Income CAGR (%)
0,Apple Inc.,389.55,97.88,148.28,299.23,116.98,25.13,30.02,2.02,-0.42,-3.14
1,Microsoft Corp.,218.44,77.74,462.07,224.72,98.36,35.60,44.85,0.49,11.19,10.08
2,"Tesla, inc.",91.98,11.55,114.34,45.70,14.30,12.72,15.68,0.40,9.51,-24.85


# 📊 Phase 4: Insights & Analysis

## 🔹 Methods & Data
- **Source**: Financial data manually extracted from the last three fiscal years (2022–2024) of 10-K filings for **Apple Inc. (AAPL)**, **Microsoft Corp. (MSFT)**, and **Tesla, Inc. (TSLA)**, sourced from the [SEC EDGAR database](https://www.sec.gov/edgar).
- **Metrics included** (in **millions USD**):  
  - Total Revenue  
  - Net Income  
  - Total Assets  
  - Total Liabilities  
  - Cash Flow from Operations (CFO)  
- **Derived metrics**:  
  - *Revenue Growth (%):* Year-over-year growth in revenue  
  - *Net Income Growth (%):* Year-over-year growth in net income  
  - *Net Profit Margin (%):* Net Income ÷ Revenue × 100  
  - *Operating CF Margin (%):* CFO ÷ Revenue × 100  
  - *Leverage (Liabilities ÷ Assets)*  
  - *CAGR (2022–2024):* 2-year compound annual growth rate for Revenue and Net Income  
- **Note**: Some values are missing (`NaN`) because they were not reported in the SEC filings (e.g., certain 2022 balance sheet values). These are intentionally left blank.

---

## 🔹 Company-Level Insights (2022–2024)

### **Apple Inc.**
- **Revenue:** Slight dip in 2023 (-2.8%), small rebound in 2024 (+2.0%).  
- **Net Income:** Flat in 2023 (+0.1%), but dropped by -6.3% in 2024.  
- **Profitability:** Very strong margins (~25–26% profit margin, ~28–31% CFO margin).  
- **Leverage:** High (≈2.0), meaning liabilities are about double assets.  
- **CAGR (2022→2024):** Revenue essentially flat (-0.42%), Net Income slightly negative (-0.08%).  

**Takeaway:** Apple remains highly profitable but shows slowing/stagnant growth and carries relatively high leverage.

---

### **Microsoft Corp.**
- **Revenue:** Consistent growth — +6.9% in 2023, +15.7% in 2024.  
- **Net Income:** Small dip in 2023 (-0.5%), then a strong rebound in 2024 (+21.8%).  
- **Profitability:** Extremely high and stable profit margin (~35%) and CFO margin (41–49%).  
- **Leverage:** Low (≈0.5), showing financial strength.  
- **CAGR (2022→2024):** Revenue +11.1%, Net Income +10.4%.  

**Takeaway:** Microsoft combines robust growth with strong profitability and a conservative balance sheet.

---

### **Tesla, Inc.**
- **Revenue:** Strong growth in 2023 (+18.8%), but nearly flat in 2024 (+0.95%).  
- **Net Income:** Grew +19.4% in 2023, then dropped sharply in 2024 (-52.7%).  
- **Profitability:** Profit margin fell from ~15% (2022–23) to ~7% in 2024.  
- **Leverage:** Moderate (~0.40), with manageable liabilities relative to assets.  
- **CAGR (2022→2024):** Revenue +9.3% growth annually, but Net Income **fell -24.8% per year** over the 2-year period.  

**Takeaway:** Tesla is still growing revenue but struggled in 2024, with profitability cut in half — a potential risk factor.

---

## 🔹 Cross-Company Comparison

- **Revenue Leaders:** Apple leads in all three years, averaging nearly **$390T (in millions USD → $390B)** over 2022–24, though its growth has stagnated.  
- **Profitability:** Microsoft dominates on margins — maintaining ~36% net profit margin and ~45% operating cash flow margin, far higher than Apple (~25%) or Tesla (~11%).  
- **Growth:** Microsoft shows the **strongest sustained growth** (double-digit CAGR in both revenue and profit). Tesla’s revenue grew but profit collapsed, while Apple stayed flat.  
- **Leverage:** Apple is highly leveraged (≈2.0 liabilities-to-assets), unlike Microsoft (~0.5) or Tesla (~0.4).  

**Overall:**  
- *Microsoft* looks healthiest: strong growth, stable profitability, low leverage.  
- *Apple* is highly profitable but heavily indebted and slowing in growth.  
- *Tesla* is volatile — rapid revenue expansion, but profits dropped sharply in 2024, raising risk concerns.  

---

In [53]:
#Part 2:
#Chatbot Design & data prep

#Mapping between possible user queries: (3-5 Questions)
#1. What was Apple's Total Revenue -> code logic


#Defining a user-based chatbot that answers financial queries for 3 of the companies using pre-determined analysis
#2 params: (user_query = question, df = financial df w derived metrics and cols). 
#returns: simple chatbots answer
def simple_chatbot(user_q, df): 

    years = ['2022','2023','2024']
    query = user_q.lower()

    #Q1: Revenue for [company] in [year]?
    if 'revenue' in query and any(y in query for y in years):
        for company in df["Company"].unique():
            if company.lower().split()[0] in query: #check for match on company name in query
                for year in years:
                    if str(year) in query: 
                        value = df.loc[
                        (df["Company"] == company) & (df["Year"] == int(year)),
                        "Total Revenue"].values
                        if value.size > 0:
                            return f"{company} reported ${value[0]:,.0f} million in revenue for {year}."
                        else: 
                            return f"Sorry, I couldn't find revenue for {company} in {year}. \nTry again with Apple, Microsoft, or Telsa across years 2022-2024."

    if "profit margin" in query:
        for company in df["Company"].unique():
            if company.lower().split()[0] in query:
                subset = df[df["Company"] == company][["Year","Net Profit Margin (%)"]]
                return subset.to_string(index=False)
    if "revenue growth" in query:
        for company in df["Company"].unique():
            if company.lower().split()[0] in query:
                subset = df[df["Company"] == company][["Year","Revenue Growth (%)"]].dropna()
                return subset.to_string(index=False)            
    if "highest revenue" in query:
        for year in [2022, 2023, 2024]:
            if str(year) in query:
                idx = df.loc[df["Year"] == year, "Total Revenue"].idxmax()
                row = df.loc[idx]
                return f"In {year}, {row['Company']} had the highest revenue: ${row['Total Revenue']:,.0f} million."

    #fallback
    return "Sorry, I can only answer predefined queries about revenue, net income, growth, and margins."

print(simple_chatbot("Show Microsoft revenue growth", df))
print(simple_chatbot("What was the highest revenue in 2023?", df))
print(simple_chatbot("What was Apple revenue in 2024?", df))                     

 Year  Revenue Growth (%)
 2023            6.882030
 2024           15.669962
In 2023, Apple Inc. had the highest revenue: $383,285 million.
Apple Inc. reported $391,035 million in revenue for 2024.
